In [1]:
import torch
from attr.validators import max_len

from model.llama_3_2 import Llama32
from model.llama_3_2_tokenizer import Llama32Tokenizer

from data.prompt_dataset import PromptedEhvoy
from data.ehovy_race import EhovyRaceDataset
from data.tokenized import TokenizedDataset

/home/lmanrique/miniconda3/envs/pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "meta-llama/Llama-3.2-1B"

In [3]:
data = EhovyRaceDataset(variation="high", split="train", max_article_size=800)
data_prompted = PromptedEhvoy(data)

In [4]:
tokenizer = Llama32Tokenizer(model_name)
model = Llama32(model_name, do_sample=False)

2025-05-10 19:42:02.560343: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-10 19:42:02.579951: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746924122.601458   25396 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746924122.608022   25396 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746924122.625448   25396 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [5]:
x, y = data_prompted[100]

In [6]:
x, y

("On May 23,1989, Stefania Follini came out from a cave at Carlsbad , New Mexico.She hadn't seen the sun for eighteen and a half weeks .Stefania was in a research program, and the scientists in the program were studying body rhythms  . In this experiment Stefania had spent 130 days in a cave ,30 feet in depth.\nDuring her time in the cave, Stefania had been completely alone except for two white mice.Her living place had been very comfortable ,but there had been nothing to feel her the time .She'd had no clock or watches , no television or radio.There had been no natural light and the temperature had always been kept at 21degC\n\nStefania stayed in the cave for a long time because  _  .\n\nA) she was asked to do research on mice\nB) she wanted to experience loneliness\nC) she was the subject of a study\nD) she needed to record her life\n\nAnswer:",
 'C')

In [7]:
x_tokenized = tokenizer(x, padding="max_length", truncation=True, return_tensors="pt", max_length=512)

In [8]:
len(x_tokenized["input_ids"][0])

512

In [9]:
import torch

In [10]:
prediction_logits = model(
    input_ids=x_tokenized["input_ids"],
    attention_mask=x_tokenized["attention_mask"],
)

In [11]:
tokens = ["A", "B", "C", "D"]
token_ids = []
for token in tokens:
    token_id = tokenizer.tokenizer.convert_tokens_to_ids(token)
    token_ids.append(token_id)

token_ids
# Note: Add space before letter to match normal word tokenization behavior

[32, 33, 34, 35]

In [12]:
probs = torch.nn.functional.softmax(prediction_logits[0, -1, token_ids], dim=-1)

In [13]:
probs

tensor([0.0081, 0.0121, 0.5054, 0.4746], device='cuda:0', dtype=torch.float16,
       grad_fn=<SoftmaxBackward0>)

In [14]:
predicted_token_id = torch.argmax(prediction_logits[0, -1, token_ids])
tokens[predicted_token_id]

'C'